In [1]:
import hashlib

from binascii import hexlify

# ECDSA Signature

Given an elliptic curve $E$ over $Z_p$ (prime $p$). Given base point $G$ of order $n$, the private key $d$, the public key $Q = d \cdot G$, and the message $m$ to be signed.

## 01. Settings

### Curve parameters
- https://en.bitcoin.it/wiki/Secp256k1

In [2]:
F = FiniteField(0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F)
C = EllipticCurve([F(0), F(7)])                                # y^2=x^3+ax+b
G = C.lift_x(0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798)
N = FiniteField(C.order())                                     # num of points on the curve

### A key pair

In [3]:
d = int(N.random_element())                                    # privkey
Q = d*G                                                        # pubkey
print(d, Q)

91646451067979531419929296935089627346081227919853588824419654463587592396732 (41650285194105644952810400568962693913058361801238541639251781599755670253544 : 433981072180906437058739893394977346989918261772728769159713782503378434551 : 1)


### Message

In [4]:
message = b'PyCon TW 2020'

## 02. Signature generation algorithm

1. Compute $e=\text{SHA}_{3}(m)$ and convert $e$ to an integer
2. Select a random integer $k$ such that $1 \le k \le n-1$
3. Compute $(x_1, y_1)=k \cdot G$
4. Convert $x_1$ to an integer and compute $r=x_1$ mod $n$. If $r=0$, return to step 2
5. Compute $s=k^{-1}(e+dr)$ mod $n$. If $s=0$, return to step 2
6. The signature for $m$ using the key $d$ is the pair $(r, s)$

In [5]:
# msg
h = hashlib.sha3_256(message).hexdigest()                      # msg
e = int(h, 16)

# sign
k = N.random_element()                                         # nonce
r = (int(k)*G).xy()[0]                                         # r = kG.x
s = (1/k)*(e+N(r)*d)                                           # s = k^-1(e+dr)

# signature
(r,s)

(54460143101901608606748136884153729283988794475785368603728342732215650943694,
 80238113962221162457703268349206538524358600215301158373357916927570924142287)

## 03. Signature verification algorithm

Given the message $m$, public key $d \cdot G$, Signature $(r, s)$

1. Compute $e=\text{SHA}_3(m)$ and convert $e$ to an integer
2. Compute $s^{-1}$ over $Z_p$
3. Compute $k_1 = e \ast s^{-1}$ mod $p$
4. Compute $k_2 = r \ast s^{-1}$ mod $p$
5. Compute $\sigma = k_1 \cdot G + k_2 \cdot Q$
6. If $\sigma = 0$, then the signature is invalid
7. $\sigma = (x, y) \in Z_p$, compute $t = x$ mod $p$
8. If $t = r$, the signature is valid

In [6]:
# verify
w = 1/N(s)                                                           # w = s^-1
r == (int(N(e)*w)*G + int(N(r)*w)*Q).xy()[0]                         # r == (e*w*G+r*w*Q).x

True

## Reference


- [1] https://en.wikipedia.org/wiki/Elliptic_Curve_Digital_Signature_Algorithm
- [2] https://github.com/TheBlueMatt/bitcoinninja/blob/master/secp256k1.ecdsa.sage